In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = pd.read_csv("final_opponent_and_team_data.csv")
data.head()

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d
0,2024-11-04,Duke,Maine,H,0.1,125.3,95.2,130.6,64.3,17.7,35.5,31.7,24-34,11-29,84.3,39.5,19.0,19.4,31.6,18-43,3-14,62,96,108.9,110.8
1,2024-11-08,Duke,Army,H,0.1,124.7,92.3,141.0,61.3,11.3,43.6,25.4,18-33,17-38,81.8,39.1,18.3,23.3,14.1,13-35,8-29,58,100,107.2,119.0
2,2024-11-12,Duke,Kentucky,N,-0.2,106.8,86.4,95.7,42.3,9.3,25.0,23.9,24-47,4-24,102.3,47.6,14.6,23.3,38.1,15-38,10-25,77,72,119.6,88.1
3,2024-11-16,Duke,Wofford,H,-0.1,124.7,56.9,133.7,61.3,15.5,45.2,17.7,14-24,16-38,54.4,28.9,29.5,29.3,5.3,9-24,5-33,35,86,69.8,115.5
4,2024-11-22,Duke,Arizona,A,0.6,111.7,75.9,101.9,50.0,20.7,35.1,21.3,17-36,9-25,81.2,45.3,22.2,16.7,20.8,15-30,6-23,55,69,98.9,90.4


In [3]:
data.dtypes

Date               object
Team               object
Opponent           object
Location           object
WAB               float64
ADJO              float64
ADJD              float64
EFF               float64
EFG%              float64
TO%               float64
OR%               float64
FTR               float64
2P                 object
3P                 object
Opp EFF           float64
Opp EFG%          float64
Opp TO%           float64
Opp OR%           float64
Opp FTR           float64
Opp 2P             object
Opp 3P             object
Opponent_score      int64
Team_score          int64
opp_adj_o         float64
opp_adj_d         float64
dtype: object

In [4]:
data.isnull().sum()

Date              0
Team              0
Opponent          0
Location          0
WAB               0
ADJO              0
ADJD              0
EFF               0
EFG%              0
TO%               0
OR%               0
FTR               0
2P                0
3P                0
Opp EFF           0
Opp EFG%          0
Opp TO%           0
Opp OR%           0
Opp FTR           0
Opp 2P            0
Opp 3P            0
Opponent_score    0
Team_score        0
opp_adj_o         0
opp_adj_d         0
dtype: int64

In [5]:
data[data.isnull().any(axis=1)]

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d


In [6]:
data = data.dropna()

In [7]:
def preprocess(data):
    data_clean = data.dropna()
    columns_to_convert = [
        "ADJO",
        "ADJD",
        "EFG%",
        "TO%",
        "OR%",
        "FTR",
        "Opp EFG%",
        "Opp TO%",
        "Opp OR%",
        "Opp FTR",
        "opp_adj_o",
        "opp_adj_d",
    ]
    for col in columns_to_convert:
        data[col] = pd.to_numeric(data[col], errors="coerce")

    ratio_columns = ["2P", "3P", "Opp 2P", "Opp 3P"]

    # Convert existing ratio columns to numerical percentages
    for column in ratio_columns:
        if column in data.columns:  # Check if the column exists in the dataset
            data[column] = (
                data[column]
                .str.split("-")
                .apply(
                    lambda x: (
                        int(x[0]) / int(x[1]) if len(x) == 2 and int(x[1]) != 0 else 0
                    )
                )
            )
    categorical_columns = ["Location"]
    data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

    data_encoded = data_encoded.apply(
        lambda x: x.astype(int) if x.dtype == "bool" else x
    )

    return data_encoded

In [8]:
preprocessed_data = preprocess(data)

In [9]:
# numeric_columns = ["Team_score", "Opponent_score"]

# # Initialize scaler
# scaler = StandardScaler()

# # Scale numeric columns
# preprocessed_data[numeric_columns] = scaler.fit_transform(preprocessed_data[numeric_columns])

In [10]:
X = preprocessed_data.drop(
    columns=[
        "Team_score",
        "Opponent_score",
        "Team",
        "Opponent",
        "2P",
        "3P",
        "Opp 2P",
        "Opp 3P",
        "WAB",
        "EFF",
        "Opp EFF",
        "Date",
    ]
)
y = preprocessed_data[["Team_score", "Opponent_score"]]

X = X.fillna(0)

In [11]:
X.dtypes

ADJO          float64
ADJD          float64
EFG%          float64
TO%           float64
OR%           float64
FTR           float64
Opp EFG%      float64
Opp TO%       float64
Opp OR%       float64
Opp FTR       float64
opp_adj_o     float64
opp_adj_d     float64
Location_H      int64
Location_N      int64
dtype: object

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [14]:
model = MLPRegressor(
    hidden_layer_sizes=(32, 16), activation="relu", max_iter=500, random_state=42
)

In [15]:
model.fit(X_train, y_train)

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)

In [16]:
y_pred_scaled = model.predict(X_test)

In [17]:
y_pred = scaler_y.inverse_transform(y_pred_scaled)

In [20]:
# Ensure that y_test is 2D
y_test = np.array(y_test)

# Check if y_test and y_pred have matching shapes
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")

# If y_test is 2D (it should be with shape (61, 2)), proceed with the metrics
if y_test.ndim == 2 and y_pred.ndim == 2:
    mae_team = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
    mae_opponent = mean_absolute_error(y_test[:, 1], y_pred[:, 1])

    mse_team = mean_squared_error(y_test[:, 0], y_pred[:, 0])
    mse_opponent = mean_squared_error(y_test[:, 1], y_pred[:, 1])

    r2_team = r2_score(y_test[:, 0], y_pred[:, 0])
    r2_opponent = r2_score(y_test[:, 1], y_pred[:, 1])

    print("Team Score Prediction Metrics:")
    print(f"MAE: {mae_team}")
    print(f"MSE: {mse_team}")
    print(f"R² Score: {r2_team}")

    print("\nOpponent Score Prediction Metrics:")
    print(f"MAE: {mae_opponent}")
    print(f"MSE: {mse_opponent}")
    print(f"R² Score: {r2_opponent}")
else:
    print("The shapes of y_test and y_pred do not match.")

Shape of y_test: (61, 2)
Shape of y_pred: (61, 2)
Team Score Prediction Metrics:
MAE: 78.97570822060536
MSE: 6377.666389291225
R² Score: -6833.628915042071

Opponent Score Prediction Metrics:
MAE: 67.47908325493877
MSE: 4677.363016395422
R² Score: -6199.785725916307


### Predicting

In [21]:
predict_data = pd.read_csv("basketball_games_data.csv")
predict_data

,date,Location,Team,Opponent,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,opp_adj_o,opp_adj_d
0,2024-12-22,Home,Oklahoma,Central Arkansas,112.6,103.1,55.9,12.3,28.6,20.6,44.5,16.5,22.0,17.2,97.5,107.7
1,2024-12-22,Away,Illinois St.,Cornell,118.1,109.0,63.0,15.1,23.3,33.3,58.5,13.6,11.1,32.1,112.8,116.8
2,2024-12-22,Neutral,Illinois,Missouri,110.9,93.4,50.9,22.9,36.4,40.4,42.2,12.1,23.1,63.8,118.0,98.2
3,2024-12-22,Home,Colgate,Iona,104.5,125.5,63.3,29.2,34.6,32.7,54.8,20.4,42.9,25.4,110.5,111.1
4,2024-12-22,Home,Alabama,Kent St.,111.2,77.8,52.5,25.0,45.5,58.3,29.3,14.5,28.6,17.3,80.8,89.7
5,2024-12-22,Home,Iowa St.,Morgan St.,117.3,107.4,66.9,14.9,30.8,44.1,49.2,17.6,27.0,23.8,112.7,114.4
6,2024-12-22,Away,New Hampshire,Stonehill,115.7,134.2,57.9,14.6,29.7,47.4,59.0,13.1,38.7,36.1,121.4,141.4
7,2024-12-22,Home,Michigan,Purdue Fort Wayne,115.8,80.5,71.2,25.7,45.0,10.2,38.5,8.6,13.3,13.8,95.2,112.1


In [22]:
predict_data["Location"] = predict_data["Location"].replace(
    {"Home": "H", "Away": "A", "Neutral": "N"}
)

In [24]:
predict_data_process = preprocess(predict_data)
predict_data_process = predict_data_process.drop(columns=["date", "Team", "Opponent"])
predict_data_process = predict_data_process.fillna(0)
predict_data_scaled = scaler_X.transform(predict_data_process)

In [25]:
predictions_scaled = model.predict(predict_data_scaled)
predictions = scaler_y.inverse_transform(predictions_scaled)

# Add predictions to the original dataset
predict_data["Predicted_Team_Score"] = predictions[:, 0]
predict_data["Predicted_Opponent_Score"] = predictions[:, 1]

# # Save the results
# new_data.to_csv("new_data_with_predictions.csv", index=False)
# print(new_data.head())

In [26]:
predict_data[["Team", "Opponent", "Predicted_Team_Score", "Predicted_Opponent_Score"]]

,Team,Opponent,Predicted_Team_Score,Predicted_Opponent_Score
0,Oklahoma,Central Arkansas,69.462060,54.802449
1,Illinois St.,Cornell,80.673212,74.562257
2,Illinois,Missouri,80.304826,77.876758
3,Colgate,Iona,81.064796,75.454935
4,Alabama,Kent St.,76.312977,50.670140
5,Iowa St.,Morgan St.,91.794663,70.808358
6,New Hampshire,Stonehill,82.453387,86.635952
7,Michigan,Purdue Fort Wayne,80.600130,40.670017
